### PoC to pull data from the Strava API, and utilize DataBricks lakehouse for storage

In [2]:
%run ./config.py

In [ ]:
#needed to grab authorization code INITIALLY
#make sure to save refresh token !
from boto import config
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id':  client_id,
    'client_secret': client_secret,
    'code': authorization_code,
    'grant_type': 'authorization_code',
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
refresh_token = res.json()['refresh_token']
access_token = res.json()['access_token']
expire_at_unix_time = res.json()['expires_at']


In [3]:
#can run this script multiple times with new refresh token
from boto import config
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'client_id':  client_id,
    'client_secret': client_secret,
    'refresh_token': new_refresh_token,
    'grant_type': 'refresh_token',
    'f': 'json'
}

res = requests.post(auth_url, data=payload, verify=False)
refresh_token = res.json()['refresh_token']
access_token = res.json()['access_token']
expire_at_unix_time = res.json()['expires_at']

In [4]:
#API call to grab all of the acitivites within a personal account
activites_url = "https://www.strava.com/api/v3/athlete/activities"
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

In [7]:
import json
import ast

import pyspark
from pyspark.sql import SparkSession

df = spark.read.json(my_dataset)

ModuleNotFoundError: No module named 'pyspark'

In [5]:
my_dataset

[{'resource_state': 2,
  'athlete': {'id': 63585039, 'resource_state': 1},
  'name': 'evening jog',
  'distance': 4801.8,
  'moving_time': 1621,
  'elapsed_time': 1632,
  'total_elevation_gain': 36.5,
  'type': 'Run',
  'sport_type': 'Run',
  'workout_type': 0,
  'id': 7815243449,
  'start_date': '2022-09-16T00:46:49Z',
  'start_date_local': '2022-09-15T17:46:49Z',
  'timezone': '(GMT-08:00) America/Los_Angeles',
  'utc_offset': -25200.0,
  'location_city': None,
  'location_state': None,
  'location_country': None,
  'achievement_count': 2,
  'kudos_count': 3,
  'comment_count': 0,
  'athlete_count': 1,
  'photo_count': 0,
  'map': {'id': 'a7815243449',
   'summary_polyline': '_bvfEn~{iUEBe@?MBa@Cc@@g@Dq@CiABg@Ga@@kAKsCB]@U?KC]@i@AGBoAE_@?EC]@a@Dq@A[Cq@A[BEAu@BSAm@@_@CcADm@AQCw@NI?KV?f@CNDb@FVLTZHTPNDh@Bf@CTH^GNDFApADTEXAT?h@C\\?VBDBb@?p@Ft@AvAAj@Bj@Cp@Dj@?PADBd@Gh@F`AE\\@\\C`@DFBRl@AHBTDFLb@Dt@APBRARBh@CdB@RETFx@Cb@OTQn@LxCDn@DLHxAAjBIn@QXIDqAHa@DQD]P_@ZEJEXH\\`@t@PRt@Xf@B|AMdCh@j@AN

In [ ]:
#get all of the activity ids from the bigger dataset
activity_ids = []
count = 0
while count < len(my_dataset):
    activity_ids.append(my_dataset[count]['id'])
    count += 1 

In [ ]:
subset_activity = activity_ids[15:35]

In [ ]:
res = [x for x in activity_ids if x not in subset_activity ]

In [ ]:
len(activity_ids)

In [ ]:
len(subset_activity)

In [ ]:
len(res)

### The activities url is the one I want to query, it is pertitent to my personal activities 
### Need to submit specific activity ID to get segment information

In [ ]:
#can get the invdividal activites
activity_id_urls = ("{}{}?include_all_efforts= True").format("https://www.strava.com/api/v3/activities/","#insertactivity")
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_activity_ = requests.get(activity_id_urls, headers=header, params=param).json()

## The distance is returned in meters

In [ ]:
my_activity_['distance']
moving_time
elapsed_time
'type'
'average_speed'
'segment_efforts'


In [ ]:
#number of segments within a particular activity 
len(my_activity_['segment_efforts'])

In [ ]:
#segment effort API call 
segment_id_url =("{}{}").format( "https://www.strava.com/api/v3/segment_efforts/","") 
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_segment_ = requests.get(segment_id_url, headers=header, params=param).json()

In [ ]:
my_segment_

In [ ]:
my_segment_['segment']['id']
#Some very interesting information in this one, oldest created date
# 'effort_count': 20871,
 #'athlete_count': 1955,
 #'star_count': 11,
segment_id_url =("{}{}").format( "https://www.strava.com/api/v3/segments/",) 
header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_segment = requests.get(segment_id_url, headers=header, params=param).json()

#### Interesting Fields from Json segment 
name': '26th St.',
 'activity_type': 'Ride',
 'distance': 674.65,
  'city': 'San Diego',



   'athlete_segment_stats': {'pr_elapsed_time': 136,
  'pr_date': '2022-04-29',
  'pr_activity_id': 7062090532,
  'effort_count': 18},
 'xoms': {'kom': '45s',
  'qom': '1:24',
  'overall': '45s',


   'effort_count': 20871,
 'athlete_count': 1955,
 'star_count': 11,

#### Helpful sparkcode from databricks cert

from pyspark.sql.functions import * 
#read in 2017 files into dataframe
df_2017 =spark.read.text(batch_2017_path)

#extract columns and appropriate headers
df_new = df_2017.withColumn('submitted_at',  col('value').substr(1,15)) \
.withColumn('order_id', col('value').substr(16,40))\
.withColumn('customer_id', col('value').substr(56,40))\
.withColumn('sales_rep_id', col('value').substr(96,40)) \
.withColumn('sales_rep_ssn', col('value').substr(136,15))\
.withColumn('sales_rep_first_name', col('value').substr(151,15))\
.withColumn('sales_rep_last_name', col('value').substr(166,15))\
.withColumn('sales_rep_address', col('value').substr(181,40))\
.withColumn('sales_rep_city', col('value').substr(221,20))\
.withColumn('sales_rep_state', col('value').substr(241,2))\
.withColumn('sales_rep_zip', col('value').substr(243,5))\
.withColumn('shipping_address_attention', col('value').substr(248,30))\
.withColumn('shipping_address_address', col('value').substr(278,40))\
.withColumn('shipping_address_city', col('value').substr(318,20))\
.withColumn('shipping_address_state', col('value').substr(338,2))\
.withColumn('shipping_address_zip', col('value').substr(340,5))\
.withColumn('product_id', col('value').substr(345,40))\
.withColumn('product_quantity', col('value').substr(385,5))\
.withColumn('product_sold_price', col('value').substr(390,20))\
.drop('value') 


df2 = df_new.select([trim(col(c)).alias(c) for c in df_new.columns]) 
df2= df2.select([when(col(c)== "", None).otherwise(col(c)).alias(c) for c in df2.columns])
df_2 = df2.withColumn("ingest_file_name", input_file_name()) \
        .withColumn("ingested_at", current_timestamp())


df_2.write.format("delta")\
.option("overwriteSchema", "true")\
.mode("overwrite")\
.save(batch_target_path)

--------------------------------------------------------------------------------------------------------------------------

#### DataBricks Code

In [ ]:
%run /config


#API call to grab all of the acitivites within a personal account
def activity_api_call(access_token):
    activites_url = "https://www.strava.com/api/v3/athlete/activities"
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': 1}
    activity_dataset = requests.get(activites_url, headers=header, params=param).json()
    
    return activity_dataset

my_dataset = activity_api_call(access_token)



#get all of the activity ids from the bigger dataset
#xtract all of the columns we want
def extract_activities(dataset):
    """Function to seperate activity_ids and create an activity dataframe. Returns 2 """
    activity_ids = []
    start_date = []
    activity_name =[]
    distance = []
    moving_time = []
    elapsed_time = []
    sport_type = []
    total_elevation_gain =[]
    count = 0
    while count < len(my_dataset):
        activity_ids.append(dataset[count]['id'])
        start_date.append(dataset[count]['start_date'])
        activity_name.append(dataset[count]['name'])
        distance.append(dataset[count]['distance'])
        moving_time.append(dataset[count]['moving_time'])
        elapsed_time.append(dataset[count]['elapsed_time'])
        sport_type.append(dataset[count]['sport_type'])
        total_elevation_gain.append(dataset[count]['total_elevation_gain'])
        count += 1 
        
    #convert list to dataframe   
    from pyspark.sql.types import LongType
    activity_id_DF = spark.createDataFrame(activity_ids, LongType())
    activity_id_DF = activity_id_DF.withColumnRenamed('value', 'activity_id')
    
    #columns names for initial DF
    #need to specify schema
    columns = ['activity_ids','start_date', 'activity_name', 'distance', 'moving_time','elapsed_time', 'sport_type'\
          ,'total_elevation_gain']
    #list of lists
    extracted_data = [activity_ids,start_date, activity_name, distance, moving_time,elapsed_time, sport_type\
          ,total_elevation_gain]

    import pandas as pd
    #create a pandas Dataframe, then convert to spark to write to storage
    pdf = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))
    activity_df = spark.createDataFrame(pdf)

    return activity_id_DF, activity_df



activity_id_DF, activity_df = extract_activities(my_dataset)

from pyspark.sql.types import LongType
#convert list to sparkdataframes
activity_id_DF = spark.createDataFrame(activity_ids, LongType())
activity_id_DF = activity_id_DF.withColumnRenamed('value', 'activity_id')


#write the activity ids to storage, overwrite the previous iteration
activity_id_DF,activity_id_path
def write_activity_ids_to_storage(dataset, storage_path):
    """Function to write activity ids to storage. Will overwrite current delta file in storage"""
    dataset.write.format("delta")\
    .option("overwriteSchema", "true")\
    .mode("overwrite")\
    .save(storage_path)

write_activity_ids_to_storage(activity_id_DF,activity_id_path)
#read the activities from storage
stored_activity_ids = spark.read.format("delta").load(activity_id_path)


#read in historical activity IDS to compare
#trying to compare current query vs historical, if nothing is written to storage ie first run, still need to execute and write original DF to storage
from pyspark.sql.utils import AnalysisException
try:
    historical_activity_ids = spark.read.format("delta").load(historical_activity_id_path)
except: 
    AnalysisException, NameError
else:
    write_activity_df_to_storage()
else:
    new_activity_ids = [x for x in activity_id_DF if x not in historical_activity_ids ]
#compare historical vs recent query,
#will then need to keep it under 15 until len(new_activity_ids) =0 


#columns names for initial DF
#need to specify schema
columns = ['activity_ids','start_date', 'activity_name', 'distance', 'moving_time','elapsed_time', 'sport_type'\
          ,'total_elevation_gain']
#list of lists
extracted_data = [activity_ids,start_date, activity_name, distance, moving_time,elapsed_time, sport_type\
          ,total_elevation_gain]

import pandas as pd
#create a pandas Dataframe, then convert to spark to write to storage
pdf = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))
activity_df = spark.createDataFrame(pdf)

#write the activity ids to storage, overwrite the previous iteration

activity_id_DF.write.format("delta")\
.option("overwriteSchema", "true")\
.mode("overwrite")\
.save(historical_activity_id_path)